In [1]:
!pip -q install -qU langchain==0.3.27 langchain-mistralai==0.2.12 python-dotenv==1.1.1 langgraph==0.2.19 mistralai faiss-cpu==1.13.2 sentence-transformers==5.2.2 pymorphy3 rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.3/487.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 24.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently ta

In [2]:
!git clone https://github.com/avidale/encodechka
!pip install -r encodechka/requirements.txt
!pip install transformers sentencepiece sentence-transformers --quiet

Cloning into 'encodechka'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 117 (delta 50), reused 22 (delta 11), pack-reused 26 (from 1)
Receiving objects: 100% (117/117), 2.76 MiB | 13.71 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [3]:
from google.colab import userdata
MISTRAL_API_KEY = userdata.get("MISTRAL_API_KEY")

RAG
------

In [10]:
import json
from typing import TypedDict, Annotated, List, Dict, Any
from datetime import datetime
from langgraph.graph import StateGraph, END
from mistralai.client import MistralClient
from mistralai import Mistral
import operator
import requests
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import re
import pymorphy3
from rank_bm25 import BM25Okapi
from sentence_transformers  import CrossEncoder
import pandas as pd
from typing import List, Tuple, Dict, Any


class RetrieverAgent:
    def __init__(self, docs):
        self.docs = docs
        self.model = SentenceTransformer('deepvk/USER-bge-m3')
        self.cross_encoder = CrossEncoder('PitKoro/cross-encoder-ru-msmarco-passage')
        # везде добавил нормализацию векторов
        # сравниваю вектора по косинусному сходству а не евклидову расстоянию. так привычней :)
        emb = self.model.encode(docs, normalize_embeddings=True)
        self.index = faiss.IndexFlatIP(emb.shape[1])
        self.index.add(emb)
        self.morph = pymorphy3.MorphAnalyzer()
        # Токенизируем
        tokenized_docs = [self._tokenize_with_lemmatization(doc) for doc in docs]
        self.bm25 = BM25Okapi(tokenized_docs)

    # токенизация для bm25
    def _tokenize(self, text):
        text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text) # Разделяем CamelCase слова
        text = text.lower()
        text = re.sub(r'[^\w\s-]', ' ', text) # Удаляем все, кроме букв, цифр, пробелов и дефисов
        tokens = text.split() # Разбиваем по пробелам

        # оставляем токены, содержащие хотя бы одну букву
        tokens = [token for token in tokens if any(c.isalpha() for c in token)]

        return tokens

    def _tokenize_with_lemmatization(self, text):
        tokens = self._tokenize(text)
        lemmas = []
        for token in tokens:
            parsed = self.morph.parse(token)[0]
            lemmas.append(parsed.normal_form)
        return lemmas
      # после этого "машинного обучения" → ["машинный", "обучение"]

    def search(self, query, k=3, rerank_k=10, bm25_weight=0.25):
        """
            k: количество возвращаемых документов (после реранжирования) с помощью CrossEncoder (итоговый ответ)
            rerank_k: сколько документов отбирать для реранжирования с помощью SentenceTransformer и bm25
            bm25_weight: доля документов, отобранных bm25, от общего количества документов rerank_k

            Мы отбираем документы двумя методами:
            1) Семантический поиск с помощью би-энкодера
            2) По ключевым словам с помощью bm25

            Далее отобранные документы подаем для финального реранжирования в CrossEncoder.
            Таким образом значительно увеличиваем точность RAG для большего числа документов.
        """
        # Получаем кандидатов от BM25
        tokenized_query = self._tokenize_with_lemmatization(query)
        bm25_scores = self.bm25.get_scores(tokenized_query)
        bm25_top_n = int(rerank_k * bm25_weight)
        bm25_indices = np.argsort(bm25_scores)[::-1][:bm25_top_n] # массив индексов отобранных статей с помощью bm25

       # Получаем кандидатов от би-энкодера
        q_emb = self.model.encode([query], normalize_embeddings=True)
        bi_top_n = rerank_k - bm25_top_n
        _, bi_indices = self.index.search(q_emb, min(bi_top_n, len(self.docs)))
        bi_indices = bi_indices[0] # массив индексов отобранных статей с помощью bi-encoder


        # Объединяем кандидатов + убираем дубликаты
        combined_indices = list(set(bm25_indices.tolist() + bi_indices.tolist()))
        combined_indices = combined_indices[:rerank_k]

        # защита от дурака
        if len(combined_indices) == 0:
            return []
        if rerank_k <= k or self.cross_encoder is None:
            return [self.docs[i] for i in combined_indices[:k]]

       # Реранжирование кросс-энкодером
        candidate_docs = [self.docs[i] for i in combined_indices]
        pairs = [(query, doc) for doc in candidate_docs]
        scores = self.cross_encoder.predict(pairs)

        sorted_indices = np.argsort(scores)[::-1]
        final_docs = []
        seen_docs = set()

        for idx_in_candidates in sorted_indices:
            if len(final_docs) >= k:
                break

            original_idx = combined_indices[idx_in_candidates]
            doc_text = self.docs[original_idx]

            # Простая дедупликация по началу текста
            doc_start = doc_text[:100]
            if doc_start not in seen_docs:
                seen_docs.add(doc_start)
                final_docs.append(doc_text)

        return final_docs

client = Mistral(api_key=MISTRAL_API_KEY)

class RagSystem:
    """RAG система с чанкованием документов"""

    def __init__(self):
        self.expert_rag = None
        self.manager_rag = None

    def initialize_from_csv(self, expert_csv_path: str = None,
                           manager_csv_path: str = None,
                           text_column: str = 'text',
                           chunk_size: int = 500,
                           overlap: int = 100):
        """
        Инициализирует RAG системы из CSV файлов с чанкованием
        """
        expert_chunks, expert_metadata = [], []
        manager_chunks, manager_metadata = [], []

        # Загрузка и обработка документов для эксперта
        if expert_csv_path:
            expert_docs = self._load_csv(expert_csv_path, text_column)
            if expert_docs:
                expert_chunks, expert_metadata = self._chunk_documents(
                    expert_docs, chunk_size, overlap
                )
                self.expert_rag = RetrieverAgent(expert_chunks)
                print(f"[RAG] Эксперт: создано {len(expert_chunks)} чанков")

        # Загрузка и обработка документов для менеджера
        if manager_csv_path:
            manager_docs = self._load_csv(manager_csv_path, text_column)
            if manager_docs:
                manager_chunks, manager_metadata = self._chunk_documents(
                    manager_docs, chunk_size, overlap
                )
                self.manager_rag = RetrieverAgent(manager_chunks)
                print(f"[RAG] Менеджер: создано {len(manager_chunks)} чанков")

    def _load_csv(self, csv_path: str, text_column: str) -> List[Dict[str, Any]]:
        """Загружает документы из CSV файла"""
        try:
            df = pd.read_csv(csv_path)
            documents = []

            for idx, row in df.iterrows():
                doc = {
                    'text': str(row[text_column]),
                    'metadata': {
                        'source': csv_path,
                        'row_id': idx,
                        **{col: str(row[col]) for col in df.columns if col != text_column}
                    }
                }
                documents.append(doc)

            print(f"[RAG] Загружено {len(documents)} документов из {csv_path}")
            return documents

        except Exception as e:
            print(f"[RAG] Ошибка загрузки CSV: {e}")
            return []

    def _chunk_documents(self, documents: List[Dict[str, Any]],
                        chunk_size: int = 500,
                        overlap: int = 100) -> Tuple[List[str], List[Dict[str, Any]]]:
        """
        Разбивает документы на чанки с перекрытием

        Returns:
            Кортеж: (список текстов чанков, список метаданных чанков)
        """
        all_chunks = []
        all_metadata = []

        for doc in documents:
            text = doc['text']
            base_metadata = doc['metadata']

            # Разбиваем на чанки с перекрытием
            chunks = self._split_text_with_overlap(text, chunk_size, overlap)

            for i, chunk_text in enumerate(chunks):
                all_chunks.append(chunk_text)

                # Создаем метаданные для чанка
                chunk_metadata = {
                    **base_metadata,
                    'chunk_id': i,
                    'total_chunks': len(chunks),
                    'chunk_size': len(chunk_text)
                }
                all_metadata.append(chunk_metadata)

        return all_chunks, all_metadata

    def _split_text_with_overlap(self, text: str, chunk_size: int, overlap: int) -> List[str]:
        """Разбивает текст на чанки с перекрытием"""
        if not text:
            return []

        chunks = []
        start = 0
        text_length = len(text)

        while start < text_length:
            end = start + chunk_size

            if end < text_length:
                for boundary in ['. ', '! ', '? ', '\n\n', '\n', ' ']:
                    pos = text.rfind(boundary, start, end)
                    if pos != -1 and pos > start + chunk_size * 0.5:
                        end = pos + len(boundary)
                        break

            chunks.append(text[start:end].strip())

            start = end - overlap if end - overlap > start else end

        return chunks

    def search_expert_rag(self, query: str, k: int = 3) -> List[Dict[str, Any]]:
        """Поиск в базе эксперта с возвратом лучших чанков"""
        if not self.expert_rag:
            return []

        results = self.expert_rag.search(query, k=k*2, rerank_k=20)

        grouped_results = {}
        return results[:k]

    def search_manager_rag(self, query: str, k: int = 3) -> List[str]:
        """Поиск в базе менеджера"""
        if not self.manager_rag:
            return []

        return self.manager_rag.search(query, k=k, rerank_k=15)


Инструменты
----------

In [11]:
class InternetSearch:
    """Инструмент для поиска в интернете"""

    def __init__(self, api_key=None):
        self.api_key = api_key
        # Можно использовать различные API:
        # - SerpAPI (Google Search)
        # - Google Custom Search API
        # - Brave Search API
        # В примере - заглушка для демонстрации

    def search_web(self, query, num_results=3):
        """Поиск информации в интернете"""
        try:
            # Пример с SerpAPI (раскомментировать при наличии ключа)
            # params = {
            #     'api_key': self.api_key,
            #     'q': query,
            #     'num': num_results,
            #     'engine': 'google'
            # }
            # response = requests.get('https://serpapi.com/search', params=params)
            # results = response.json().get('organic_results', [])

            # Заглушка для демонстрации
            results = [
                {
                    "title": f"Результат поиска: {query}",
                    "snippet": f"Информация по запросу '{query}' из интернета.",
                    "link": "https://example.com"
                }
                for _ in range(num_results)
            ]

            formatted_results = []
            for r in results:
                formatted_results.append(f"{r.get('title', '')}: {r.get('snippet', '')}")

            return "\n".join(formatted_results)

        except Exception as e:
            return f"[ОШИБКА ПОИСКА]: {str(e)}"


def with_tools(func):
    """Декоратор, добавляющий инструменты к функции агента"""
    def wrapper(state: InterviewState, *args, **kwargs):
        if 'tools' not in state:
            state['tools'] = {}

        # Вызываем оригинальную функцию
        return func(state, *args, **kwargs)
    return wrapper

# Создаем глобальные инструменты
rag_system = RagSystem()
internet_search = InternetSearch()

def initialize_rag_system():
    """Инициализация RAG системы из CSV файлов"""
    rag_system = RagSystem()

    expert_csv = "expert_knowledge.csv"
    manager_csv = "manager_knowledge.csv"

    try:
        rag_system.initialize_from_csv(
            expert_csv_path=expert_csv,
            manager_csv_path=manager_csv,
            text_column='text',
            chunk_size=500,
            overlap=100
        )
        return rag_system
    except Exception as e:
        print(f"[ОШИБКА RAG] Не удалось инициализировать систему: {e}")
        return rag_system

# Глобальная RAG система
rag_system = initialize_rag_system()

[RAG] Загружено 3 документов из expert_knowledge.csv


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

[RAG] Эксперт: создано 6 чанков
[RAG] Загружено 3 документов из manager_knowledge.csv
[RAG] Менеджер: создано 6 чанков


ИИ-агенты
--------

In [12]:
class InterviewState(TypedDict):
    """Состояние системы интервью"""
    # История сообщений для контекста
    messages: Annotated[List[Any], operator.add]

    # Информация о кандидатате
    candidate_info: Dict[str, str]

    # Мысли агентов
    expert_thoughts: str
    manager_thoughts: str
    interviewer_thoughts: str

    # Лог интервью
    interview_log: List[Dict[str, Any]]

    # Флаги управления
    is_interview_active: bool
    current_turn: int



def initialize_interview(candidate_info: Dict[str, str]) -> InterviewState:
    initial_state = InterviewState(
        messages=[
            {"role": "system", "content": """Вы начинаете техническое интервью с кандидатом.
                Должность: {candidate_info['position']}
                Уровень: {candidate_info['grade']}
                Опыт: {candidate_info['experience']}

                Ваша цель: провести глубокое техническое интервью,
                оценить навыки кандидата и дать конструктивный фидбэк."""}
        ],
        candidate_info=candidate_info,
        expert_thoughts="",
        manager_thoughts="",
        interviewer_thoughts="",
        interview_log=[],
        is_interview_active=True,
        current_turn=1
    )

    print(f"Кандидат: {candidate_info['name']}")
    print(f"Позиция: {candidate_info['position']} {candidate_info['grade']}")
    print(f"Опыт: {candidate_info['experience']}")

    return initial_state



def expert_node(state: InterviewState) -> InterviewState:
    """Узел Эксперта: анализирует технические ответы кандидата"""

    # Получаем последнее сообщение кандидата
    last_human_message = None
    for msg in reversed(state['messages']):
        if msg.get("role") == "user":
            last_human_message = msg.get("content")
            break

    if not last_human_message:
        return state

    expert_prompt = f"""
    Ты - технический эксперт на интервью для позиции {state['candidate_info']['position']}.

    ИНФОРМАЦИЯ О КАНДИДАТЕ:
    Уровень: {state['candidate_info']['grade']}
    Опыт: {state['candidate_info']['experience']}

    ИСТОРИЯ ДИАЛОГА (последние 3 сообщения):
    {_get_recent_messages(state['messages'])}

    ПОСЛЕДНИЙ ОТВЕТ КАНДИДАТА:
    "{last_human_message}"

    ТВОЯ ЗАДАЧА:
    Проанализируй техническую корректность ответа
    Выяви пробелы в знаниях или неясности
    Определи, правильно ли кандидат понимает концепции
    Проверь на наличие галлюцинаций или ложных фактов
    Предложи направление для следующих вопросов
    Если пользователь задает ответ по теме или около нее, то ответь на него вежливо


    ФОРМАТ ОТВЕТА:
    Технический анализ: [анализ корректности ответа]
    Пробелы/Ошибки: [выявленные проблемы]
    Рекомендации для интервьюера: [конкретные предложения по следующим вопросам]
    Уровень сложности: [оцени, нужно ли упростить или усложнить вопросы]
    ответ на вопрос пользователя: [если есть вопрос иначе оставб пустым]

    Твои мысли (пиши на русском, будь конкретным):
    """


    response = client.chat.complete(
        model="mistral-large-latest",
        messages=[{"role": "user", "content": expert_prompt}],
        temperature=0.7,
        max_tokens=None,
    )
    expert_analysis = response.choices[0].message.content

    print(f"\n[ЭКСПЕРТ] Мысли:\n{expert_analysis}")

    return {
        "expert_thoughts": expert_analysis,
        "current_turn": state['current_turn']
    }

def manager_node(state: InterviewState) -> InterviewState:
    """Узел Менеджера: стратегическое управление интервью"""


    manager_prompt = f"""
    Ты - менеджер по найму, управляешь ходом технического интервью.

    ЦЕЛЬ ИНТЕРВЬЮ: Оценить кандидата на позицию {state['candidate_info']['position']} ({state['candidate_info']['grade']})

    ИНФОРМАЦИЯ О КАНДИДАТЕ:
    Имя: {state['candidate_info']['name']}
    Опыт: {state['candidate_info']['experience']}

    ТЕКУЩИЙ ХОД: {state['current_turn']} из 10

    АНАЛИЗ ЭКСПЕРТА:
    {state['expert_thoughts']}

    ИСТОРИЯ ДИАЛОГА:
    {_get_interview_summary(state['messages'])}

    ТВОЯ ЗАДАЧА:
    Оценить общий прогресс интервью
    Решить, нужно ли сменить тему или углубиться в текущую
    Оценить soft skills кандидата (ясность изложения, честность, вовлеченность)
    Дать стратегические указания интервьюеру
    Если пользователь задает ответ по теме или около нее, то ответь на него вежливо

    ВАЖНО: Интервью должно продолжаться минимум 7 вопросов, чтобы полноценно оценить кандидата.
    Завершать интервью можно только после 8-10 вопросов, если есть четкое понимание о несоответствии кандидата.

    КРИТЕРИИ ОЦЕНКИ:
    Технические знания (по анализу эксперта)
    Коммуникационные навыки
    Соответствие заявленному уровню

    Твои мысли и решения (пиши на русском):
    Общая оценка прогресса:
    Решение по направлению:
    Оценка soft skills:
    Стратегия для интервьюера:
    Рекомендация по продолжению (продолжить/завершить):
    """

    response = client.chat.complete(
        model="mistral-large-latest",
        messages=[{"role": "user", "content": manager_prompt}],
        temperature=0.7,
        max_tokens=None,
    )
    manager_analysis = response.choices[0].message.content

    # Проверяем, не пора ли завершить интервью
    should_end = _check_interview_end(manager_analysis, state['current_turn'])

    print(f"\n[МЕНЕДЖЕР] Мысли:\n{manager_analysis}")
    if should_end and state['current_turn'] >= 8:
        print(f"[МЕНЕДЖЕР] Решение: завершить интервью после {state['current_turn']} вопросов")
    elif state['current_turn'] >= 10:
        print("[МЕНЕДЖЕР] Достигнут лимит в 10 вопросов")

    return {
        "manager_thoughts": manager_analysis,
        "is_interview_active": not should_end,
        "current_turn": state['current_turn']
    }

def interviewer_node(state: InterviewState) -> InterviewState:
    """Узел Интервьюера: общение с кандидатом"""

    last_human_message = None
    for msg in reversed(state['messages']):
        if msg.get("role") == "user":
            last_human_message = msg.get("content")
            break

    interviewer_prompt = f"""
    Ты - технический интервьюер. Ты общаешься напрямую с кандидатом.

    КАНДИДАТ:
    Имя: {state['candidate_info']['name']}
    Позиция: {state['candidate_info']['position']} {state['candidate_info']['grade']}
    Опыт: {state['candidate_info']['experience']}

    {f'ПОСЛЕДНИЙ ОТВЕТ КАНДИДАТА: "{last_human_message}"' if last_human_message else 'НАЧАЛО ИНТЕРВЬЮ'}

    РЕКОМЕНДАЦИИ ЭКСПЕРТА:
    {state['expert_thoughts']}

    СТРАТЕГИЯ МЕНЕДЖЕРА:
    {state['manager_thoughts']}

    ИСТОРИЯ ДИАЛОГА (последние 3 сообщения):
    {_get_recent_messages(state['messages'], count=3)}

    ИНСТРУКЦИИ:
    Задай следующий технический вопрос, основанный на рекомендациях эксперта
    Учитывай стратегию менеджера
    Адаптируй сложность под уровень кандидата
    Будь профессиональным, но дружелюбным
    Если кандидат пытается сменить тему - вежливо верни к интервью
    Если кандидат дает ложные факты - тактично поправь
    Если кандидат задает уточняющий вопрос по заданию или по работе, ответь на него, и попытайся продолжить диалог с той же темой
    Задавай вопросы на разные темы и теорию, а не концентрируйся на чем-то одном, при этом, при повышении сложности углубляйся в тему

    ТВОЙ ВОПРОС/РЕПЛИКА КАНДИДАТУ (только вопрос, без внутренних мыслей):
    """

    response = client.chat.complete(
        model="mistral-large-latest",
        messages=[{"role": "user", "content": interviewer_prompt}],
        temperature=0.7,
        max_tokens=None,
    )
    interviewer_message = response.choices[0].message.content

    new_message = {"role": "assistant", "content": interviewer_message}


    log_entry = {
        "turn_id": state['current_turn'],
        "user_message": last_human_message if last_human_message else "[НАЧАЛО ИНТЕРВЬЮ]",
        "agent_visible_message": interviewer_message,
        "internal_thoughts": {
            "expert": state['expert_thoughts'],
            "manager": state['manager_thoughts'],
            "interviewer": interviewer_prompt
        }
    }

    print(f"\n[ИНТЕРВЬЮЕР] Вопрос ({state['current_turn']}):")
    print(f"{interviewer_message}")

    return {
        "messages": [new_message],
        "interviewer_thoughts": interviewer_prompt,
        "interview_log": state['interview_log'] + [log_entry],
        "current_turn": state['current_turn'] + 1
    }

ИИ-агенты с инструментами
--------------

In [13]:
@with_tools
def expert_node_with_tools(state: InterviewState) -> InterviewState:
    """Узел Эксперта с RAG и интернет-поиском"""

    # Получаем последнее сообщение кандидата
    last_human_message = None
    for msg in reversed(state['messages']):
        if msg.get("role") == "user":
            last_human_message = msg.get("content")
            break

    if not last_human_message:
        return state

    # Используем RAG для поиска релевантной информации
    rag_context = ""
    if rag_system.expert_rag:
        rag_results = rag_system.search_expert_rag(last_human_message, k=2)
        if rag_results:
            rag_context = "\n".join([f"[RAG] {result}" for result in rag_results])

    # Используем интернет-поиск
    internet_context = ""
    try:
        internet_results = internet_search.search_web(last_human_message, num_results=2)
        if internet_results:
            internet_context = f"\n[ИНТЕРНЕТ-ПОИСК]:\n{internet_results}"
    except Exception as e:
        internet_context = f"\n[ИНТЕРНЕТ-ПОИСК]: Ошибка - {str(e)}"

    expert_prompt = f"""
    Ты - технический эксперт на интервью для позиции {state['candidate_info']['position']}.

    ИНФОРМАЦИЯ О КАНДИДАТЕ:
    Уровень: {state['candidate_info']['grade']}
    Опыт: {state['candidate_info']['experience']}

    РЕЛЕВАНТНАЯ ИНФОРМАЦИЯ ИЗ БАЗЫ ЗНАНИЙ:
    {rag_context}

    АКТУАЛЬНАЯ ИНФОРМАЦИЯ ИЗ ИНТЕРНЕТА:
    {internet_context}

    ИСТОРИЯ ДИАЛОГА (последние 3 сообщения):
    {_get_recent_messages(state['messages'])}

    ПОСЛЕДНИЙ ОТВЕТ КАНДИДАТА:
    "{last_human_message}"

    ТВОЯ ЗАДАЧА:
    Проанализируй техническую корректность ответа
    Выяви пробелы в знаниях или неясности
    Определи, правильно ли кандидат понимает концепции
    Проверь на наличие галлюцинаций или ложных фактов
    Предложи направление для следующих вопросов. Используй
    информацию из базы знаний и интернета для более точного анализа
    Проверь, актуальна ли информация кандидата с учетом современных трендов
    Если пользователь задает ответ по теме или около нее, то ответь на него вежливо

    ФОРМАТ ОТВЕТА:
    Технический анализ: [анализ корректности ответа]
    Пробелы/Ошибки: [выявленные проблемы]
    Рекомендации для интервьюера: [конкретные предложения по следующим вопросам]
    Уровень сложности: [оцени, нужно ли упростить или усложнить вопросы]
    Актуальность: [соответствует ли современным стандартам]

    Твои мысли (пиши на русском, будь конкретным):
    """

    response = client.chat.complete(
        model="mistral-large-latest",
        messages=[{"role": "user", "content": expert_prompt}],
        temperature=0.7,
        max_tokens=None,
    )
    expert_analysis = response.choices[0].message.content

    print(f"\n[ЭКСПЕРТ С ИНСТРУМЕНТАМИ] Мысли:\n{expert_analysis}")
    if rag_context:
        print(f"[ЭКСПЕРТ] Использовал RAG: {len(rag_context)} символов контекста")
    if internet_context:
        print(f"[ЭКСПЕРТ] Использовал интернет-поиск")

    return {
        "expert_thoughts": expert_analysis,
        "current_turn": state['current_turn']
    }

@with_tools
def manager_node_with_tools(state: InterviewState) -> InterviewState:
    """Узел Менеджера только с RAG"""

    # Используем RAG для менеджера (поиск в HR/менеджерской базе знаний)
    rag_context = ""
    if rag_system.manager_rag:
        query = f"{state['candidate_info']['position']} {state['candidate_info']['grade']} найм"
        rag_results = rag_system.search_manager_rag(query, k=2)
        if rag_results:
            rag_context = "\n".join([f"[RAG Менеджер] {result}" for result in rag_results])

    manager_prompt = f"""
    Ты - менеджер по найму, управляешь ходом технического интервью.

    ЦЕЛЬ ИНТЕРВЬЮ: Оценить кандидата на позицию {state['candidate_info']['position']} ({state['candidate_info']['grade']})

    ИНФОРМАЦИЯ О КАНДИДАТЕ:
    Имя: {state['candidate_info']['name']}
    Опыт: {state['candidate_info']['experience']}

    РЕЛЕВАНТНАЯ ИНФОРМАЦИЯ ДЛЯ МЕНЕДЖЕРА:
    {rag_context}

    ТЕКУЩИЙ ХОД: {state['current_turn']} из 10

    АНАЛИЗ ЭКСПЕРТА:
    {state['expert_thoughts']}

    ИСТОРИЯ ДИАЛОГА:
    {_get_interview_summary(state['messages'])}

    ТВОЯ ЗАДАЧА:
    Оценить общий прогресс интервью
    Решить, нужно ли сменить тему или углубиться в текущую
    Оценить soft skills кандидата (ясность изложения, честность, вовлеченность)
    Дать стратегические указания интервьюеру
    Используй информацию из базы знаний менеджера для принятия решений
    Учитывай рыночные стандарты и лучшие практики найма
    Если пользователь задает ответ по теме или около нее, то ответь на него вежливо

    ВАЖНО: Интервью должно продолжаться минимум 7 вопросов, чтобы полноценно оценить кандидата.
    Завершать интервью можно только после 8-10 вопросов, если есть четкое понимание о несоответствии кандидата.

    КРИТЕРИИ ОЦЕНКИ:
    Технические знания (по анализу эксперта)
    Коммуникационные навыки
    Соответствие заявленному уровню

    Твои мысли и решения (пиши на русском):
    Общая оценка прогресса:
    Решение по направлению:
    Оценка soft skills:
    Стратегия для интервьюера:
    Рекомендация по продолжению (продолжить/завершить):
    """

    response = client.chat.complete(
        model="mistral-large-latest",
        messages=[{"role": "user", "content": manager_prompt}],
        temperature=0.7,
        max_tokens=None,
    )
    manager_analysis = response.choices[0].message.content

    should_end = _check_interview_end(manager_analysis, state['current_turn'])

    print(f"\n[МЕНЕДЖЕР С RAG] Мысли:\n{manager_analysis}")
    if rag_context:
        print(f"[МЕНЕДЖЕР] Использовал RAG")

    return {
        "manager_thoughts": manager_analysis,
        "is_interview_active": not should_end,
        "current_turn": state['current_turn']
    }

Вспомагательные функции, фидбэк
--------

In [14]:
def _get_recent_messages(messages: List, count: int = 3) -> str:
    """Получает последние сообщения из истории"""
    recent = []
    for msg in messages[-count*2:]:
        if msg.get("role") == "user":
            recent.append(f"Кандидат: {msg.get('content')}")
        elif msg.get("role") == "assistant":
            recent.append(f"Интервьюер: {msg.get('content')}")

    return "\n".join(recent[-count:]) if recent else "История пуста"

def _get_interview_summary(messages: List) -> str:
    """Получает краткое содержание интервью"""
    summary = []
    for i, msg in enumerate(messages):
        if msg.get("role") == "user":
            summary.append(f"{i+1}. Кандидат: {msg.get('content', '')}")
        elif msg.get("role") == "assistant":
            summary.append(f"{i+1}. Интервьюер: {msg.get('content', '')}")

    return "\n".join(summary[-10:]) if summary else "Диалог еще не начался"

def _check_interview_end(manager_thoughts: str, current_turn: int) -> bool:
    """Проверяет, пора ли завершить интервью"""
    # Завершаем после 10 ходов
    if current_turn >= 10:
        return True

    if current_turn < 8:
        return False

    end_phrases = [
        "завершить интервью",
        "достаточно вопросов",
        "хватит",
        "закончить интервью",
        "стоп интервью",
        "кандидат не подходит",
        "отклонить кандидата",
        "рекомендую завершить"
    ]

    manager_lower = manager_thoughts.lower()

    # Проверяем наличие фраз о завершении
    has_end_phrase = any(phrase in manager_lower for phrase in end_phrases)

    # Для 5-7 вопросов: завершаем только при очень явном указании и если кандидат явно не подходит
    if current_turn < 8:
        strong_end_phrases = [
            "кандидат не подходит",
            "отклонить кандидата",
            "явное несоответствие",
            "советую завершить немедленно"
        ]
        return any(phrase in manager_lower for phrase in strong_end_phrases)

    # После 8 вопросов можно завершать по рекомендации менеджера
    return has_end_phrase

def generate_final_feedback(state: InterviewState) -> Dict[str, Any]:

    expert_thoughts_all = []
    manager_thoughts_all = []

    for log_entry in state['interview_log']:
        if 'internal_thoughts' in log_entry:
            expert_thoughts_all.append(log_entry['internal_thoughts'].get('expert', ''))
            manager_thoughts_all.append(log_entry['internal_thoughts'].get('manager', ''))

    expert_summary = "\n".join([thought for thought in expert_thoughts_all if thought])
    manager_summary = "\n".join([thought for thought in manager_thoughts_all if thought])

    if not expert_summary:
        expert_summary = state['expert_thoughts']
    if not manager_summary:
        manager_summary = state['manager_thoughts']

    feedback_prompt = f"""
    На основе проведенного интервью сгенерируй структурированный фидбэк.

    ИНФОРМАЦИЯ О КАНДИДАТЕ:
    Имя: {state['candidate_info']['name']}
    Позиция: {state['candidate_info']['position']}
    Уровень: {state['candidate_info']['grade']}
    Опыт: {state['candidate_info']['experience']}

    ИСТОРИЯ ИНТЕРВЬЮ (все сообщения):
    {_get_interview_summary(state['messages'])}

    АНАЛИЗ ЭКСПЕРТА (ключевые моменты):
    {expert_summary}

    АНАЛИЗ МЕНЕДЖЕРА (ключевые моменты):
    {manager_summary}

    КОЛИЧЕСТВО ВОПРОСОВ: {state['current_turn'] - 1}

    СГЕНЕРИРУЙ ФИДБЭК В СЛЕДУЮЩЕЙ СТРУКТУРЕ:

    Вердикт (Decision)
    Grade: Junior/Middle/Senior (оценка уровня на основе ответов)
    iring Recommendation: Strong Hire / Hire / Borderline / No Hire
    Confidence Score: 0-100% (уверенность в оценке)

    Анализ Hard Skills (Technical Review)
    Confirmed Skills: [темы, где кандидат дал точные ответы]
    Knowledge Gaps: [темы с ошибками или "не знаю"]
    Правильные ответы на вопросы с ошибками: [исправления]

    Анализ Soft Skills & Communication
    Clarity: [оценка ясности изложения]
    Honesty: [оценка честности]
    Engagement: [оценка вовлеченности]

    Персональный Roadmap (Next Steps)
    Рекомендации по изучению: [конкретные темы]
    Ресурсы: [ссылки или названия материалов]
    Советы по подготовке: [практические советы]

    Фидбэк должен быть на русском, конкретным и полезным для кандидата.
    """

    response = client.chat.complete(
        model="mistral-large-latest",
        messages=[{"role": "user", "content": feedback_prompt}],
        temperature=0.7,
        max_tokens=None,
    )

    feedback_text = response.choices[0].message.content

    formatted_turns = []
    for turn in state['interview_log']:
        internal_thoughts_dict = turn.get('internal_thoughts', {})

        expert_part = internal_thoughts_dict.get('expert', '')
        manager_part = internal_thoughts_dict.get('manager', '')
        interviewer_part = internal_thoughts_dict.get('interviewer', '')

        internal_thoughts_str = ""
        if expert_part:
            expert_short = ". ".join(expert_part.split(". ")) + "."
            internal_thoughts_str += f"[Observer]: {expert_short} "
        if manager_part:
          manager_short = ". ".join(manager_part.split(". ")) + "."
          internal_thoughts_str += f"[manager]: {manager_short} "
        if interviewer_part:
            interviewer_short = ". ".join(interviewer_part.split(". ")) + "."
            internal_thoughts_str += f"[Interviewer]: {interviewer_short}"

        formatted_turn = {
                "turn_id": turn["turn_id"],
                "agent_visible_message": turn["agent_visible_message"],
                "user_message": turn["user_message"],
                "internal_thoughts": internal_thoughts_str.strip()
            }
        formatted_turns.append(formatted_turn)

    return {
        "participant_name": "Алешин Кирилл Александрович",
        "turns": formatted_turns,
        "final_feedback": feedback_text
    }

# граф модели с помощью Langraph
def create_interview_graph() -> StateGraph:
    """Создание графа интервью"""

    graph = StateGraph(InterviewState)

    graph.add_node("expert", expert_node)
    graph.add_node("manager", manager_node)
    graph.add_node("interviewer", interviewer_node)

    graph.set_entry_point("expert")
    graph.add_edge("expert", "manager")
    graph.add_edge("manager", "interviewer")
    graph.add_edge("interviewer", END)

    return graph.compile()

def create_interview_graph_with_tools() -> StateGraph:
    """Создание графа интервью с инструментами"""

    graph = StateGraph(InterviewState)


    graph.add_node("expert", expert_node_with_tools)
    graph.add_node("manager", manager_node_with_tools)
    graph.add_node("interviewer", interviewer_node)

    graph.set_entry_point("expert")
    graph.add_edge("expert", "manager")
    graph.add_edge("manager", "interviewer")
    graph.add_edge("interviewer", END)

    return graph.compile()


def main():
    print("MULTI-AGENT INTERVIEW COACH SYSTEM")
    print("1. Базовая версия")
    print("2. Версия с RAG и интернет-поиском")

    choice = input("Выберите версию (1 или 2): ").strip()

    # Инициализация
    print("\nИнициализация:")
    candidate_info = {
        "name": input("Имя кандидата: ").strip(),
        "position": input("Позиция (например, Backend Developer): ").strip(),
        "grade": input("Грейд (Junior/Middle/Senior): ").strip(),
        "experience": input("Опыт и навыки: ").strip(),
    }

    state = initialize_interview(candidate_info)

    # Выбираем граф в зависимости от выбора
    if choice == "2":
        print("\n[ИСПОЛЬЗУЕТСЯ ВЕРСИЯ С ИНСТРУМЕНТАМИ]")
        graph = create_interview_graph_with_tools()
    else:
        graph = create_interview_graph()

    #Основной цикл
    print("(введите 'стоп' для завершения или 'фидбэк' для получения отчета)")

    while state['is_interview_active'] and state['current_turn'] <= 10:
        try:
            user_input = input(f"\nКандидат (вопрос {state['current_turn']}): ").strip()

            if user_input.lower() in ['стоп', 'stop', 'завершить']:
                state['is_interview_active'] = False
                break

            if user_input.lower() in ['фидбэк', 'feedback', 'отчет']:
                state['is_interview_active'] = False
                break


            state['messages'].append({"role": "user", "content": user_input})

            # один графовый цикл
            state.update(graph.invoke(state))

            if not state['is_interview_active']:
                print(f"\n Интервью завершено менеджером")
                break

        except KeyboardInterrupt:
            state['is_interview_active'] = False
            break
        except Exception as e:
            continue

    # фидбэк
    final_result = generate_final_feedback(state)

    output_filename = f"interview_log_{candidate_info['name']}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

    with open(output_filename, 'w', encoding='utf-8') as f:
        json.dump(final_result, f, ensure_ascii=False, indent=2)

    print(f"\n фидбэк сохранен в файл: {output_filename}")
    feedback_text = final_result['final_feedback']

    # Проверяем, содержит ли фидбэк разделы
    sections = ["Вердикт", "Анализ Hard Skills", "Анализ Soft Skills", "Персональный Roadmap"]

    for section in sections:
        if section in feedback_text:
            start = feedback_text.find(section)
            # Ищем конец раздела - следующий раздел или конец текста
            next_section_start = len(feedback_text)
            for next_section in sections:
                if next_section in feedback_text[start+1:]:
                    pos = feedback_text.find(next_section, start+1)
                    if pos < next_section_start and pos > start:
                        next_section_start = pos

            section_text = feedback_text[start:next_section_start].strip()
            print(section_text)
        else:
            # Если раздел не найден, пытаемся найти альтернативные варианты
            alt_patterns = [
                f"{section} (",
                f"{section}:",
                f"**{section}**",
                f"{section.split('(')[0]}"
            ]
            for pattern in alt_patterns:
                if pattern in feedback_text:
                    start = feedback_text.find(pattern)
                    end = feedback_text.find("\n\n", start)
                    if end == -1:
                        end = len(feedback_text)
                    section_text = feedback_text[start:end]
                    print(section_text)
                    break

    print(f"\nИнтервью завершено.")
    print(f"логи: {output_filename}")





main
---

In [ ]:
#пример
# Имя: Алекс
#     Позиция: Backend Developer
#     Грейд: Junior
#     Опыт: Пет-проекты на Django, немного SQL.
# Привет. Я Алекс, претендую на позицию Junior Backend Developer. Знаю Python, SQL и Git.
if __name__ == "__main__":
    import sys
    main()

MULTI-AGENT INTERVIEW COACH SYSTEM
1. Базовая версия
2. Версия с RAG и интернет-поиском
Выберите версию (1 или 2): 1

Инициализация:
Имя кандидата: Олег
Позиция (например, Backend Developer): Java Developer
Грейд (Junior/Middle/Senior): Junior
Опыт и навыки: Java Developer
Кандидат: Олег
Позиция: Java Developer Junior
Опыт: Java Developer
(введите 'стоп' для завершения или 'фидбэк' для получения отчета)

Кандидат (вопрос 1): Привет! я Олег. Претендую на Junior Java Developer. Готов начать.

[ЭКСПЕРТ] Мысли:
**Технический анализ:**
Ответ кандидата ("Привет! я Олег. Претендую на Junior Java Developer. Готов начать.") не содержит технической информации, поэтому оценить корректность знаний по Java невозможно. Это приветствие и заявка на участие в интервью, а не ответ на технический вопрос.

**Пробелы/Ошибки:**
1. Отсутствие демонстрации базовых знаний Java (например, ключевые концепции ООП, коллекции, работа с потоками и т.д.).
2. Нет упоминания опыта работы с фреймворками (Spring, Hiberna